In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import copy
import datetime
import os
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from okada_wrapper import dc3dwrapper

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [3]:
import celeri
celeri = reload(celeri)

In [4]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [5]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

Isolating slip rate constraints
- Dip-slip rate constraint on Nevada02abb: rate = -2.40 (mm/yr), 1-sigma uncertainty = +/-1.20 (mm/yr)
- Strike-slip rate constraint on SierraConnectbb: rate = -1.70 (mm/yr), 1-sigma uncertainty = +/-0.80 (mm/yr)
- Strike-slip rate constraint on TestSjfa: rate = -17.50 (mm/yr), 1-sigma uncertainty = +/-8.80 (mm/yr)
- Strike-slip rate constraint on aaa: rate = -3.50 (mm/yr), 1-sigma uncertainty = +/-1.70 (mm/yr)
- Strike-slip rate constraint on asdf: rate = -4.00 (mm/yr), 1-sigma uncertainty = +/-2.00 (mm/yr)
- Strike-slip rate constraint on asdf: rate = 2.80 (mm/yr), 1-sigma uncertainty = +/-2.50 (mm/yr)
- Strike-slip rate constraint on asdfb: rate = -9.20 (mm/yr), 1-sigma uncertainty = +/-4.60 (mm/yr)
- Strike-slip rate constraint on asdfba: rate = -17.00 (mm/yr), 1-sigma uncertainty = +/-8.50 (mm/yr)
- Dip-slip rate constraint on asdfbb: rate = -3.00 (mm/yr), 1-sigma uncertainty = +/-1.50 (mm/yr)
- Strike-slip rate constraint on cfm_camp_rock_complete_

In [6]:
success, u, grad_u = dc3dwrapper(0.6, [1.0, 1.0, -1.0], 3.0, 90, [-0.7, 0.7], [-0.7, 0.7], [1.0, 0.0, 0.0]) 

In [7]:
def get_utm_zone(lon, lat):
    """
    Determine UTM zone for a longitude and latitude point.
    From: https://pyproj4.github.io/pyproj/stable/examples.html
    Also see: https://pyproj4.github.io/pyproj/stable/api/aoi.html#pyproj.aoi.AreaOfInterest
    This appears pretty slow.  Should I consider:
    https://github.com/Turbo87/utm
    """
    if lon > 180.0:
        lon = lon - 360

    utm_crs_list = pyproj.database.query_utm_crs_info(
        datum_name="WGS 84",
        area_of_interest=pyproj.aoi.AreaOfInterest(
            west_lon_degree=lon,
            south_lat_degree=lat,
            east_lon_degree=lon,
            north_lat_degree=lat,
        ),
    )
    utm_zone = utm_crs_list[0].name[-3:]
    return utm_zone

# function G = GetElasticPartials(F, S)
"""
Calculates the elastic displacement partial derivatives based on the Okada
formulation, using the source and receiver geometries defined in
dicitonaries segment and stations. Before calculating the partials for
each segment, a local oblique Mercator project is done.
"""

n_stations = len(station)
n_segments = len(segment)
segment_strike = np.zeros(n_segments)
G = np.zeros((3 * n_stations, 3 * n_segments))
v1 = np.zeros(n_segments)
v2 = np.zeros(n_segments)
v3 = np.zeros(n_segments)

# Loop through each segment and calculate displacements
# parfor (i = 1:nSeg)
for i in range(1):
    utm_zone = get_utm_zone(segment.mid_lon[i], segment.mid_lat[i])
    print(i, utm_zone)
#     f = structsubset(F, i);
#     % Calculate projected Cartesian coordinates
#     [f, s] = ProjectSegCoords(f, S)
#     segment_strike[i] = f.strike;

#     % Calculate fault parameters in Okada form
#     [strike, L, W, ofx, ofy, ofxe, ofye, ...
#      tfx, tfy, tfxe, tfye] = fault_params_to_okada_form(f.px1, f.py1, f.px2, f.py2, deg_to_rad(f.dip), f.lDep, f.bDep);

#     % Displacements due to unit slip components
#     [ves vns vus...
#      ved vnd vud...
#      vet vnt vut] = okada_partials(ofx, ofy, strike, f.lDep, deg_to_rad(f.dip), L, W, 1, 1, 1, s.fpx, s.fpy, command.poissons_ratio);
     
#     v1{i} = reshape([ves vns vus]', 3*nSta, 1);
#     v2{i} = reshape(sign(90 - f.dip).*[ved vnd vud]', 3*nSta, 1);
#     v3{i} = reshape((f.dip - 90 == 0).*[vet vnt vut]', 3*nSta, 1); 
#     v1{i} = xyz2enumat((v1{i}), -f.strike + 90);
#     v2{i} = xyz2enumat((v2{i}), -f.strike + 90);
#     v3{i} = xyz2enumat((v3{i}), -f.strike + 90);

# Place cell arrays into the partials matrix
# G[:, 0::3) = cell2mat(v1)
# G[:, 1::3] = cell2mat(v2)
# G[:, 2::3] = cell2mat(v3) 


0 12N


In [8]:
# from pyproj import Proj
# projection = Proj(proj="utm", zone=32, ellps="WGS84")
utm_zone = "9"
projection = pyproj.Proj(proj="utm", zone=utm_zone, ellps="WGS84")
x, y = projection(station.lon, station.lat)
recovered_lon, recovered_lat = projection(x, y, inverse=True)
recovered_lon[np.where(recovered_lon < 0)] += 360

# plt.figure()
# plt.plot(station.lon, station.lat, ".r")
# plt.show()

# plt.figure()
# plt.plot(recovered_lon - station.lon)
# plt.show()

In [31]:
# Consider https://github.com/Turbo87/utm
# Because it has latitude bands which proj does not.

# pyproj oblique mercator notes
# latitude_projection_centre (float) – Latitude of projection centre (lat_0).
# longitude_projection_centre (float) – Longitude of projection centre (lonc).
# azimuth_initial_line (float) – Azimuth of initial line (azimuth).
# angle_from_rectified_to_skew_grid (float) – Angle from Rectified to Skew Grid (gamma).

# The 4th argument is obscure and possibly should always be 90 degrees?
# http://geotiff.maptools.org/proj_list/hotine_oblique_mercator.html



# Pyproj oblique mercator
# https://proj.org/operations/projections/omerc.html
# The following two lines work and we're keeping them around for reference
# crs = pyproj.CRS.from_proj4("+proj=omerc +lon_1=-1 +lat_1=1 +lon_2=0 +lat_2=0 +ellps=WGS84")
# projection = pyproj.Proj(crs)

i = 0
lon1 = segment.lon1[i]
lon2 = segment.lon2[i]
lat1 = segment.lat1[i]
lat2 = segment.lat2[i]

if lon1 > 180.0:
    lon1 = lon1 - 360
if lon2 > 180.0:
    lon2 = lon2 - 360
projection_string = ("+proj=omerc "
                    + "+lon_1=" + str(lon1) + " "
                    + "+lat_1=" + str(lat1) + " "
                    + "+lon_2=" + str(lon2) + " "
                    + "+lat_2=" + str(lat2) + " "
                    + "+ellps=WGS84")
projection = pyproj.Proj(pyproj.CRS.from_proj4(projection_string))

station_lon_projected_x, station_lat_projected_y = projection(station.lon, station.lat)
segment_lon1_projected_x, segment_lat1_projected_y = projection(segment.lon1[i], segment.lat1[i])
segment_lon2_projected_x, segment_lat2_projected_y = projection(segment.lon2[i], segment.lat2[i])


plt.figure()
plt.plot(station.lon, station.lat, '.b', markersize=1)
plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
plt.show()

plt.figure()
plt.plot([segment_lon1_projected_x, segment_lon2_projected_x], [segment_lat1_projected_y, segment_lat2_projected_y], "-r")
plt.plot(station_lon_projected_x, station_lat_projected_y, '.b', markersize=1)
plt.show()

# p = pyproj.crs.coordinate_operation.HotineObliqueMercatorBConversion(100, 5, 0, 0)
# print(type(p))
# print(type(projection))
# projection(station.lon, station.lat)

# According to: https://proj.org/operations/projections/omerc.html
# This is this already rotated by the fault strike but the rotation can be undone with +no_rot
"""
+no_rot
No rectification (not “no rotation” as one may well assume).
Do not take the last step from the skew uv-plane to the map XY plane.

Note: This option is probably only marginally useful, but remains for (mostly) historical reasons.
"""


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/var/folders/fl/sm71y44x3tqc7r1wc40k10gc0000gn/T/ipykernel_46792/3880934897.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

'\n+no_rot\nNo rectification (not “no rotation” as one may well assume).\nDo not take the last step from the skew uv-plane to the map XY plane.\n\nNote: This option is probably only marginally useful, but remains for (mostly) historical reasons.\n'

In [16]:
str(lon1)

'-112.91300000000001'